# Trabalho Final da disciplina Linguística Computacional

Aluno: Ariel Augusto dos Santos

Data: 27/09/21

## Tema: Geração de Paráfrases

Neste trabalho, treinamos um modelo transformer para tranformar sentenças arbitrárias em paráfrases aproximandamente equivalentes. A geração automática de paráfrases de uma determinada frase é uma tarefa importante porém desafiadora no processamento de linguagem natural (PNL) \[[Li et al., 2018](https://aclanthology.org/D18-1421/)\].

Utilizei a biblioteca de NLP [Hugging Face](https://huggingface.co) e o modelo de linguagem desenvolvido pelo Google [T5](https://arxiv.org/abs/1910.10683).

Este notebook é um passo a passo do código e demonstração do trabalho desenvolvido. O relatório final está disponível [aqui]().

## Setup

Dependências Python:
- 🤗 Datasets
- [simpleT5](https://github.com/Shivanandroy/simpleT5)


#### simpleT5
simpleT5 é um *wrapper* em torno das bibliotecas `🤗 Transformers` e `PyTorch lightning` que facilita o treinamento e inferência de modelos T5.

In [1]:
! pip install datasets simplet5 numpy

import numpy as np
import pandas as pd


## Dataset

Para treinamento do parafraseador, usamos o [TaPaCo Corpus](https://zenodo.org/record/3707949#.X9Dh0cYza3I):
> a freely available paraphrase corpus for 73 languages extracted from the Tatoeba database

O Tatoeba é um banco online de sentenças voltado para estudantes de idiomas estrangeiros, criado no Japão.

In [2]:
from datasets import load_dataset

paraphrase_dataset = load_dataset("tapaco", "pt")

paraphrases_data = paraphrase_dataset['train'].remove_columns(
    ['sentence_id', 'lists', 'tags', 'language']
).to_pandas()
print('Nº de sentenças:', len(paraphrases_data))

paraphrase_groups = paraphrases_data.groupby(['paraphrase_set_id'])
print('Nº de sentenças equivalentes:', len(paraphrase_groups.groups))


Reusing dataset tapaco (/root/.cache/huggingface/datasets/tapaco/pt/1.0.0/71d200534b520a174927a8f0479c06220a0a6fb5201a84ebfce19006c6354698)


  0%|          | 0/1 [00:00<?, ?it/s]

Nº de sentenças: 78430
Nº de sentenças equivalentes: 29949


### Preparação e split de datasets

Realizamos o fine-tuning do T5 passando duas sentenças: a sequência original e a saída desejada. Utilizei um split 10/90 para avaliação e treinamento, com 29949 entradas totais.

Separo as entradas do dataset original em grupos de paráfrases equivalentes e então crio tuplas de sentenças equivalentes a partir de uma janela deslizante. Se necessário, é adicionado uma cópia da primeira sentença no grupo para completar a janela final.

In [3]:
import os.path
from tqdm.autonotebook import tqdm

tqdm.pandas()


def datasets():
    """
    generates tuples consisting of source_text and target_text
    returns (evaluation set, training set)
    """
    if os.path.isfile("tapaco-pairs-pt.csv"):
        sentence_pairs = pd.read_csv("tapaco-pairs-pt.csv")
    else:
        sentence_pairs = pd.DataFrame(columns=["source_text", "target_text"])
        for group_key in tqdm(paraphrase_groups.groups.keys()):
            paraphrases = paraphrase_groups.get_group(
                group_key)["paraphrase"].tolist()
            if len(paraphrases) % 2 != 0:
                paraphrases = np.pad(paraphrases, (0, 1),
                                     constant_values=paraphrases[-1])
            for idx, sentence in enumerate(paraphrases[:len(paraphrases) // 2]):
                sentence_pairs = sentence_pairs.append({
                    "source_text": "paraphrase: "+sentence,
                    "target_text": paraphrases[-(idx + 1)]
                }, ignore_index=True)

        sentence_pairs.to_csv("tapaco-pairs-pt.csv")

    test = sentence_pairs.sample(frac=0.1)
    train = sentence_pairs.drop(test.index)

    return (test, train)


test_df, train_df = datasets()


## Modelo de Linguagem: T5

Utilizamos a arquitetura *Text-To-Text Transfer Transformer* (T5), um transformer "clássico" desenvolvido no Google e descrito em *"Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer"* (Raffel et al., 2019).

O modelo específico que utilizei foi o mT5, uma versão massivamente multilíngue do T5 descrito em "mT5: A Massively Multilingual Pre-trained Text-to-Text Transformer" (Xue et al., 2021). O mT5 foi treinado com 101 linguagens e o checkpoint usado neste trabalho (mT5-base) possui 580 milhões de parâmetros.

### Treinamento

* Como o mT5 ocupa muita memória, é necessário limitar o tamanho das sequências de entrada
* O treinamento converge rapidamente, provavelmente devido ao dataset, mas é necessário mais experimentação com os hiperparâmetros

In [ ]:
from simplet5 import SimpleT5
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

model = SimpleT5()
model.from_pretrained("mt5", "google/mt5-base")

model.train(
    train_df=train_df,
    eval_df=test_df,
    max_epochs=1,  # em apenas 1 época já converge
    batch_size=6,  # variar de acordo com memória disponível
    source_max_token_len=72,  # devido ao tamanho dos embeddings multilíngues,
    target_max_token_len=72,  # é necessário limitar o tamanho das sequências
    use_gpu=True
)


Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                        | Params
------------------------------------------------------
0 | model | MT5ForConditionalGeneration | 582 M 
------------------------------------------------------
582 M     Trainable params
0         Non-trainable params
582 M     Total params
2,329.605 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
Global seed set to 42
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]